In [221]:
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair

import json

In [222]:
BDG_ROOT_URL = '35.157.161.130'
BDB_URL_PORT = 9984
bdb = BigchainDB('http://{}:{}'.format(BDG_ROOT_URL, BDB_URL_PORT))

## Creacion de Assets

In [223]:
# Alice es un usuario definido por un par de claves
alice = generate_keypair()

# bob tbn
bob = generate_keypair()

In [224]:
alice_secret = generate_keypair()

In [143]:
# Alice's vote
vote = {
    'data': {
        'vote': {
            # lo que hay aqui se puede cambiar
            'identifier': 'question_14102017',
            'manufacturer': True,
        },
    }, 
}

In [144]:
# Creacion del asset

# create asset
prepared_creation_tx = bdb.transactions.prepare(
        operation='CREATE',
        signers=alice.public_key,
        asset=bicycle,
)

fulfilled_creation_tx = bdb.transactions.fulfill(
    prepared_creation_tx, 
    private_keys=alice.private_key, # signed with private key
)

# send asset
sent_creation_tx = bdb.transactions.send(fulfilled_creation_tx)

txid = fulfilled_creation_tx['id'] # id de la transaccion

In [145]:
fulfilled_creation_tx

{'asset': {'data': {'sida': {'cuanto_sida': ['mu', 'cho'],
    'identifier': 'pamtumaca',
    'manufacturer': 'bkfab'}}},
 'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586',
 'inputs': [{'fulfillment': 'pGSAIPFKQ_qXpUSijT_z7M46BYUx0t_zUD7pm4j_1lqBMxbcgUAybT0Fx1YVm_GyqxO0WtpXn1oxDX66WX3IQzHHPoDV9suGGaAOoTyHFWj96t7UAIpDswBGsHmvvKoz1D2I1XAO',
   'fulfills': None,
   'owners_before': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
 'metadata': None,
 'operation': 'CREATE',
 'outputs': [{'amount': '1',
   'condition': {'details': {'public_key': 'HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9',
     'type': 'ed25519-sha-256'},
    'uri': 'ni:///sha-256;LikjEvHldymOvitUiFkBNEWwjkc6ygt4OaO2b5tlEtM?fpt=ed25519-sha-256&cost=131072'},
   'public_keys': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
 'version': '1.0'}

In [146]:
creation_tx = bdb.transactions.retrieve(txid)

In [147]:
alice.public_key

'HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9'

In [148]:
# Alice is the owner
creation_tx['outputs'][0]['public_keys'][0] == alice.public_key

True

In [150]:
# Buscar bicis por serial number:
assets = bdb.assets.get(search="pamtumaca")
assets

[{'data': {'sida': {'cuanto_sida': ['mu', 'cho'],
    'identifier': 'pamtumaca',
    'manufacturer': 'bkfab'}},
  'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'}]

## Transferencia de Assets

In [179]:
fulfilled_creation_tx['id']

'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'

In [180]:
# Alice le quiere enviar la bici con id
bici = assets[0]
bici['id']

'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'

In [174]:
# Notar que la bici es de alice
creation_tx = bdb.transactions.retrieve(bici['id'])
creation_tx['outputs'][0]['public_keys'][0] == alice.public_key

True

In [181]:
transfer_asset = {
    'id': bici['id'],
}

In [195]:
output_index = 0 # selecion de la opciones de output
output = creation_tx['outputs'][output_index]
transfer_input = {
    'fulfillment': output['condition']['details'],
    'fulfills': {
        'output_index': output_index,
        'transaction_id': creation_tx['id'],
    },
    'owners_before': output['public_keys'],
}

prepared_transfer_tx = bdb.transactions.prepare(
    operation='TRANSFER',
    asset=transfer_asset,
    inputs=transfer_input,
    recipients=alice_secret.public_key,
)

fulfilled_transfer_tx = bdb.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=alice.private_key,
)

In [196]:
sent_transfer_tx = bdb.transactions.send(fulfilled_transfer_tx)

BadRequest: (400, '{\n  "message": "Invalid transaction (DoubleSpend): input `f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586` was already spent", \n  "status": 400\n}\n', {'message': 'Invalid transaction (DoubleSpend): input `f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586` was already spent', 'status': 400})

In [209]:
sent_transfer_tx

{'asset': {'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'},
 'id': '5cfba500c357bef3243e694e4b5afa3c104f4402c86b608df4b8cf5e074185a4',
 'inputs': [{'fulfillment': 'pGSAIPFKQ_qXpUSijT_z7M46BYUx0t_zUD7pm4j_1lqBMxbcgUA1hBW_JNLF8AcsO6O7Yv72ZnocyVIh8nQvwDanWnRKE-rYEvNeF7IOVtmhLIEKovx-pTlOnusatG_-xbE6n1gH',
   'fulfills': {'output_index': 0,
    'transaction_id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'},
   'owners_before': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
 'metadata': None,
 'operation': 'TRANSFER',
 'outputs': [{'amount': '1',
   'condition': {'details': {'public_key': 'DcpriYX58N1NqyXBkA7WpsV7FKMwqwhvcdmuSGEQxKYW',
     'type': 'ed25519-sha-256'},
    'uri': 'ni:///sha-256;pmwlr6fe-TdVjHEDBlEGwa90VLXvx8isHNevqv_Q2cI?fpt=ed25519-sha-256&cost=131072'},
   'public_keys': ['DcpriYX58N1NqyXBkA7WpsV7FKMwqwhvcdmuSGEQxKYW']}],
 'version': '1.0'}

In [211]:
bdb.transactions.status('5cfba500c357bef3243e694e4b5afa3c104f4402c86b608df4b8cf5e074185a4')

{'status': 'valid'}

In [198]:
bob.public_key

'DcpriYX58N1NqyXBkA7WpsV7FKMwqwhvcdmuSGEQxKYW'

In [218]:
search = bdb.assets.get(search='pamtumaca')

In [220]:
bdb.transactions.get(asset_id="f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586")

[{'asset': {'data': {'sida': {'cuanto_sida': ['mu', 'cho'],
     'identifier': 'pamtumaca',
     'manufacturer': 'bkfab'}}},
  'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586',
  'inputs': [{'fulfillment': 'pGSAIPFKQ_qXpUSijT_z7M46BYUx0t_zUD7pm4j_1lqBMxbcgUAybT0Fx1YVm_GyqxO0WtpXn1oxDX66WX3IQzHHPoDV9suGGaAOoTyHFWj96t7UAIpDswBGsHmvvKoz1D2I1XAO',
    'fulfills': None,
    'owners_before': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
  'metadata': None,
  'operation': 'CREATE',
  'outputs': [{'amount': '1',
    'condition': {'details': {'public_key': 'HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9',
      'type': 'ed25519-sha-256'},
     'uri': 'ni:///sha-256;LikjEvHldymOvitUiFkBNEWwjkc6ygt4OaO2b5tlEtM?fpt=ed25519-sha-256&cost=131072'},
    'public_keys': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
  'version': '1.0'},
 {'asset': {'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'},
  'id': '5cfba500c357bef3243e694e4b5afa3c104f4402

In [212]:
bdb.transactions.retrieve('5cfba500c357bef3243e694e4b5afa3c104f4402c86b608df4b8cf5e074185a4')

{'asset': {'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'},
 'id': '5cfba500c357bef3243e694e4b5afa3c104f4402c86b608df4b8cf5e074185a4',
 'inputs': [{'fulfillment': 'pGSAIPFKQ_qXpUSijT_z7M46BYUx0t_zUD7pm4j_1lqBMxbcgUA1hBW_JNLF8AcsO6O7Yv72ZnocyVIh8nQvwDanWnRKE-rYEvNeF7IOVtmhLIEKovx-pTlOnusatG_-xbE6n1gH',
   'fulfills': {'output_index': 0,
    'transaction_id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'},
   'owners_before': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
 'metadata': None,
 'operation': 'TRANSFER',
 'outputs': [{'amount': '1',
   'condition': {'details': {'public_key': 'DcpriYX58N1NqyXBkA7WpsV7FKMwqwhvcdmuSGEQxKYW',
     'type': 'ed25519-sha-256'},
    'uri': 'ni:///sha-256;pmwlr6fe-TdVjHEDBlEGwa90VLXvx8isHNevqv_Q2cI?fpt=ed25519-sha-256&cost=131072'},
   'public_keys': ['DcpriYX58N1NqyXBkA7WpsV7FKMwqwhvcdmuSGEQxKYW']}],
 'version': '1.0'}

In [207]:
alice.public_key

'HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9'

In [203]:
bob.public_key

'DcpriYX58N1NqyXBkA7WpsV7FKMwqwhvcdmuSGEQxKYW'